# The Dataset
The dataset you will use in this chapter is the Sonar dataset. This is a dataset that describes
sonar chirp returns bouncing off different surfaces. The 60 input variables are the strength of
the returns at different angles. It is a binary classification problem that requires a model to
differentiate rocks from metal cylinders.
It is a well-understood dataset. All the variables are continuous and generally in the
range of 0 to 1. The output variable is a string “M” for mine and “R” for rock, which will need
to be converted to integers 1 and 0 as neural networks can only output numbers.

A benefit of using this dataset is that it is a standard benchmark problem. This means
that we have some idea of the expected skill of a good model. Using cross-validation, a neural
network should be able to achieve a performance of around 84% with an upper bound on
accuracy for custom models at around 88%. You can learn more about this dataset on the
UCI Machine Learning repository.

# Baseline Neural Network Model performance

In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [3]:
# load dataset
dataframe = pd.read_csv("sonar.all-data", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [6]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

You are now ready to create your neural network model using Keras. You will use scikit-learn
to evaluate the model using stratified k-fold cross-validation. This is a resampling technique
that will provide an estimate of the performance of the model. It does this by splitting the
data into k parts and training the model on all parts except one, which is held out as a test set
to evaluate the performance of the model. This process is repeated k times, and the average
score across all constructed models is used as a robust estimate of performance. It is stratified,
meaning that it will look at the output values and attempt to balance the number of instances
that belong to each class in the k splits of the data.

To use Keras models with scikit-learn, you must use the `KerasClassifier wrapper` from
the SciKeras module. This class takes a function that creates and returns our neural network
model. It also takes arguments that it will pass along to the call to `fit()`, such as the number
of epochs and the batch size.

Let’s start by defining the function that creates your baseline model. Your model will
have a single, fully connected hidden layer with the same number of neurons as input variables.
This is a good default starting point when creating neural networks.

The weights are initialized using a small Gaussian random number. The Rectifier
activation function is used. The output layer contains a single neuron in order to make
predictions. It uses the sigmoid activation function in order to produce a probability output
in the range of 0 to 1 that can easily and automatically be converted to crisp class values.

Finally, you will use the logarithmic loss function `(binary_crossentropy)` during training,
the preferred loss function for binary classification problems. The model also uses the efficient
Adam optimization algorithm for gradient descent, and accuracy metrics will be collected
when the model is trained.

In [9]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_shape=(60,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Now, it is time to evaluate this model using stratified cross-validation in the scikit-learn
framework. Pass the number of training epochs to the `KerasClassifier`, again using reasonable
default values. Verbose output is also turned off, given that the model will be created ten
times for the 10-fold cross-validation being performed.

In [10]:
# evaluate model with standardized dataset
estimator = KerasClassifier(model=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

2022-09-29 22:58:22.053030: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-29 22:58:22.053104: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-29 22:58:22.053138: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (theia-20220907-182918): /proc/driver/nvidia/version does not exist
2022-09-29 22:58:22.053557: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Baseline: 82.24% (6.34%)


# Improve the Baseline Model with Data Preparation
It is a good practice to prepare your data before modeling. Neural network models are especially
suitable for having consistent input values, both in scale and distribution. Standardization is
an effective data preparation scheme for tabular data when building neural network models.
This is where the data is rescaled such that the mean value for each attribute is 0, and
the standard deviation is 1. This preserves Gaussian and Gaussian-like distributions while
normalizing the central tendencies for each attribute. You can use scikit-learn to perform the
standardization of your sonar dataset using the `StandardScaler` class.

Rather than performing the standardization on the entire dataset, it is good practice to
train the standardization procedure on the training data within the pass of a cross-validation
run, and use the trained standardization instance to prepare the unseen test fold. This makes
standardization a step in model preparation in the cross-validation process. It prevents the
algorithm from having knowledge of unseen data during evaluation, knowledge that might be
passed from the data preparation scheme like a crisper distribution.

You can achieve this in scikit-learn using a `Pipeline` class. The pipeline is a wrapper that
executes one or more models within a pass of the cross-validation procedure. Here, you can
define a pipeline with the `StandardScaler` followed by your neural network model.

In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [3]:
# load dataset
dataframe = pd.read_csv("sonar.all-data", header=None)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_shape=(60,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# evaluate baseline model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', 
    KerasClassifier(model=create_baseline, 
        epochs=100, batch_size=5, verbose=0)))

pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=10, shuffle=True)

results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

2022-09-30 02:23:36.510443: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-30 02:23:36.510578: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-30 02:23:36.510629: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (theia-20220907-182918): /proc/driver/nvidia/version does not exist
2022-09-30 02:23:36.511620: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Standardized: 87.55% (6.76%)


# Tuning Layers and Number of Neurons in the Model

There are many things to tune on a neural network, such as weight initialization, activation
functions, optimization procedure, and so on. One aspect that may have an outsized effect is
the structure of the network itself, called the network topology. In this section, you will look at two experiments on the structure of the network: making it smaller and making it larger. These are good experiments to perform when tuning a neural network on your problem.

## Evaluate a Smaller Network
Note that there is likely a lot of redundancy in the input variables for this problem. The data
describes the same signal from different angles. Perhaps some of those angles are more relevant
than others. So you can force a type of feature extraction by the network by restricting the
representational space in the first hidden layer.

In this experiment, you will take your baseline model with 60 neurons in the hidden layer
and reduce it by half to 30. This will pressure the network during training to pick out the
most important structure in the input data to model.

You will also standardize the data as in the previous experiment with data preparation
and try to take advantage of the slight lift in performance.

In [5]:
# smaller model
def create_smaller():
    # create model
    model = Sequential()
    model.add(Dense(30, input_shape=(60,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_smaller,
epochs=100, batch_size=5, verbose=0)))

pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=10, shuffle=True)

results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 83.19% (7.13%)


## Evaluate a Larger Network
A neural network topology with more layers offers more opportunities for the network to
extract key features and recombine them in useful nonlinear ways. You can easily evaluate
whether adding more layers to the network improves the performance by making another
small tweak to the function used to create our model. Here, you add one new layer (one line)
to the network that introduces another hidden layer with 30 neurons after the first hidden
layer.

The idea here is that the network is given the opportunity to model all input variables before
being bottlenecked and forced to halve the representational capacity, much like you did in the
experiment above with the smaller network. Instead of squeezing the representation of the
inputs themselves, you have an additional hidden layer to aid in the process.

In [6]:
# larger model
def create_larger():
# create model
    model = Sequential()
    model.add(Dense(60, input_shape=(60,), activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_larger, epochs=100, batch_size=5, verbose=0)))

pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=10, shuffle=True)

results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 86.10% (7.59%)
